<a href="https://colab.research.google.com/github/chw8207/Transformer/blob/main/%ED%86%A0%ED%81%AC%EB%82%98%EC%9D%B4%EC%A0%80%26%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B%EB%A7%A4%EC%B9%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim
import nltk
nltk.download('punkt')

In [50]:
import tensorflow as tf
import math
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')
%matplotlib inline

### GPU 설정

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0' :
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8530136995046180449
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 40129593344
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7252778135895904899
physical_device_desc: "device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0"
xla_global_id: 416903419
]


### Word2Vec 토큰화

In [51]:
# 데이터 토큰화하고 word2vec 모델을 훈련시키기
sample = open('/content/drive/MyDrive/Colab Notebooks/Transformer/data/kant.txt', 'r')
s = sample.read()

In [52]:
# 이스케이프 문자 전처리
f = s.replace('\n', ' ')
data = []

# 문장 파싱(parshing)
for i in sent_tokenize(f) :
  temp = []
  # 문장을 단어로 토큰화하기
  for j in word_tokenize(i) :
    temp.append(j.lower())
  data.append(temp)

In [60]:
# Skip Gram 모델 생성하기
# window: 입력 문장에서 현재 단어와 예측 단어 사이의 거리를 제한함.
# model2 = gensim.models.Word2Vec(data, min_count=1, vector_size=512,
#                                 window=5, sg=1)
model2 = Word2Vec(data, min_count=1, vector_size=512,
                                window=5, sg=1)
print(model2)

Word2Vec<vocab=10236, vector_size=512, alpha=0.025>


In [64]:
# 코싸인 유사도
def similarity(word1, word2) :
  cosine = False  # 기본값
  try :
    a = model2.wv[word1]
    cosine=True
  except KeyError :
    print(word1, ':[unk] key not found in dictionary')
  try :
    b = model2.wv[word2] # a=True is implied
  except KeyError :
    cosine=False # a, b 둘 다 true여야 함.
    print(word2, '[unk] key not not found in dictionary')

  if(cosine==True) :
    b = model2.wv[word2]
    # 코싸인 유사도 계산
    dot = np.dot(a, b)
    norma = np.linalg.norm(a)
    normb = np.linalg.norm(b)
    cos = dot / (norma * normb)

    aa = a.reshape(1, 512)
    ba = b.reshape(1, 512)
    # print('Word1', aa)
    # print('Word2', ba)

    cos_lib = cosine_similarity(aa, ba)
    # print(cos_lib, 'word_similarity')

  if(cosine==False) :
    cos_lib = 0
  return cos_lib

In [65]:
# 사례0 : 데이터셋 및 사전의 단어
word1 = 'freedom'
word2 = 'liberty'
print("Similarity",similarity(word1,word2),word1,word2)

Similarity [[0.34205425]] freedom liberty


In [66]:
# 사례1 : 데이터셋이나 사전에 없는 단어
word1 = 'corporations'
word2 = 'right'
print("Similarity",similarity(word1,word2),word1,word2)

corporations :[unk] key not found in dictionary
Similarity 0 corporations right


In [69]:
# 노이즈성 관계
word1 = 'etext'
word2 = 'declaration'
print("Similarity",similarity(word1,word2),word1,word2)

etext :[unk] key not found in dictionary
Similarity 0 etext declaration
